In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi
import torch, sys, platform
print("Py:", sys.version)
print("Plat:", platform.platform())

Mon Sep 22 07:49:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             40W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
# 0) YOLO 설치
!pip -q install ultralytics==8.3.197 lap>=0.5.12

from ultralytics import YOLO
PLAYER_MODEL = "/content/drive/MyDrive/Little_kid_0912/pose/player/player_detect/weights/best.pt"  # 네 가중치
player_model = YOLO(PLAYER_MODEL)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
!pip install hydra-core==1.3.2 omegaconf==2.3.0

!pip install --upgrade torchvision

!pip install decord

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 131.4 MB/s eta 0:00:00


In [ ]:
# ================================
# ✅ SAM2 패키지 경로 등록 & 설치
# ================================
import sys, os

SAM2_PATH = "/content/drive/MyDrive/sam2"
sys.path.append(SAM2_PATH)

!pip install -e /content/drive/MyDrive/sam2

# ================================
# ✅ 라이브러리 import
# ================================
import torch, os, cv2, numpy as np
from collections import deque
from ultralytics import YOLO
from sam2.build_sam import build_sam2_video_predictor

# ------------------------
# 준비
# ------------------------
def best_dtype():
    return torch.bfloat16 if torch.cuda.get_device_capability()[0] >= 8 else torch.float16
dtype = best_dtype()

torch.backends.cudnn.benchmark = True
try:
    torch.set_float32_matmul_precision("high")
except Exception:
    pass

os.chdir("/content/drive/MyDrive/sam2")
ckpt = "/content/drive/MyDrive/sam2/checkpoints/sam2.1_hiera_large.pt"
cfg  = "configs/sam2.1/sam2.1_hiera_l.yaml"
pred = build_sam2_video_predictor(cfg, ckpt, vos_optimized=False)

MP4 = "/content/drive/MyDrive/Little_kid_0912/Video_data/0710_night2.mp4"
with torch.inference_mode(), torch.autocast("cuda", dtype=dtype):
    state = pred.init_state(MP4)

PLAYER_MODEL = "/content/drive/MyDrive/Little_kid_0912/pose/player/player_detect/weights/best.pt"
BALL_MODEL   = "/content/drive/MyDrive/Little_kid_0912/pose/ball/ball_800/train/weights/best.pt"
player_model = YOLO(PLAYER_MODEL)
ball_model   = YOLO(BALL_MODEL)

try:
    player_model.fuse()
    ball_model.fuse()
except Exception:
    pass
player_model.to("cuda")
ball_model.to("cuda")

# ------------------------
# Util 함수
# ------------------------
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0]); yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2]); yB = min(boxA[3], boxB[3])
    inter = max(0.0, xB - xA) * max(0.0, yB - yA)
    areaA = max(0.0, (boxA[2] - boxA[0])) * max(0.0, (boxA[3] - boxA[1]))
    areaB = max(0.0, (boxB[2] - boxB[0])) * max(0.0, (boxB[3] - boxB[1]))
    denom = areaA + areaB - inter + 1e-6
    return inter / denom

def box_center(box):
    x1,y1,x2,y2 = box
    return (0.5*(x1+x2), 0.5*(y1+y2))

id2color, ema_masks, class_of = {}, {}, {}
def color_for(oid: int):
    if oid not in id2color:
        rng = np.random.default_rng(oid + 12345)
        id2color[oid] = tuple(int(c) for c in rng.integers(60,255,size=3))
    return id2color[oid]

# ------------------------
# 초기 씨딩
# ------------------------
dev = torch.device("cuda")
next_oid = 0
class_of = {}
found_players = False
found_ball = False

SCAN_MAX = 300
STEP     = 1
CONF_P   = 0.30
CONF_B   = 0.04
IMGSZ_B  = 1536

cap = cv2.VideoCapture(MP4)

def seed_object(fi, xyxy, label, use_box=True):
    global next_oid
    x1, y1, x2, y2 = xyxy
    oid = next_oid
    with torch.inference_mode(), torch.autocast("cuda", dtype=dtype):
        if use_box:
            box_t = torch.tensor([[x1, y1, x2, y2]], device=dev, dtype=torch.float32)
            xc, yc = (x1+x2)/2.0, (y1+y2)/2.0
            pts  = torch.tensor([[xc, yc]], device=dev, dtype=torch.float32)
            labs = torch.tensor([1],        device=dev, dtype=torch.long)
            pred.add_new_points_or_box(
                state, fi, oid,
                points=pts, labels=labs, box=box_t,
                normalize_coords=True
            )
        else:
            xc, yc = (x1+x2)/2.0, (y1+y2)/2.0
            pts  = torch.tensor([[xc, yc]], device=dev, dtype=torch.float32)
            labs = torch.tensor([1],        device=dev, dtype=torch.long)
            pred.add_new_points_or_box(
                state, fi, oid,
                points=pts, labels=labs, normalize_coords=True
            )
    class_of[oid] = label
    next_oid += 1
    return oid

seed_logs = []

for fi in range(0, SCAN_MAX, STEP):
    ok, bgr = cap.read()
    if not ok: break

    if not found_players:
        rp = player_model(bgr, conf=CONF_P, verbose=False)[0]
        if rp.boxes is not None and len(rp.boxes) > 0:
            confs = rp.boxes.conf.detach().float().cpu().numpy()
            order = np.argsort(-confs)[:10]
            for idx in order:
                b = rp.boxes.xyxy[idx].detach().cpu().numpy().tolist()
                oid = seed_object(fi, b, "player", use_box=True)
                seed_logs.append((oid, "player", fi))
            found_players = True

    if not found_ball:
        rb = ball_model(bgr, conf=CONF_B, iou=0.4, imgsz=IMGSZ_B,
                        agnostic_nms=True, verbose=False)[0]
        if rb.boxes is not None and len(rb.boxes) > 0:
            idx = int(rb.boxes.conf.argmax().detach().cpu())
            b = rb.boxes.xyxy[idx].detach().cpu().numpy().tolist()
            oid = seed_object(fi, b, "ball", use_box=True)
            seed_logs.append((oid, "ball", fi))
            found_ball = True

    if found_players and found_ball:
        break

cap.release()

print("초기 씨딩 완료:",
      f"players={sum(1 for _,lbl,_ in seed_logs if lbl=='player')},",
      f"ball={sum(1 for _,lbl,_ in seed_logs if lbl=='ball')}")

# ------------------------
# 비디오 루프
# ------------------------
OUT_PATH = "/content/drive/MyDrive/Little_kid_0912/result/final_output.mp4"
cap = cv2.VideoCapture(MP4)
W,H,fps = int(cap.get(3)), int(cap.get(4)), cap.get(5) or 30
out = cv2.VideoWriter(OUT_PATH, cv2.VideoWriter_fourcc(*"mp4v"), fps,(W,H))

ball_hist = deque(maxlen=8)
def px_per_frame_speed():
    if len(ball_hist) < 2: return 0.0
    (f1, x1, y1), (f2, x2, y2) = ball_hist[-2], ball_hist[-1]
    dt = max(1, f2-f1)
    return ((x2-x1)**2 + (y2-y1)**2)**0.5 / dt

with torch.inference_mode(), torch.autocast("cuda", dtype=dtype):
    for fi, obj_ids, masks in pred.propagate_in_video(state):
        cap.set(cv2.CAP_PROP_POS_FRAMES, fi)
        ok, bgr = cap.read()
        if not ok: break
        overlay = bgr.copy()

        speed = px_per_frame_speed()
        reseed_interval = 5 if speed > 25 else 15

        # ---------------- Incremental Player Seeding ----------------
        if fi % reseed_interval == 0:
            res_p = player_model(bgr, conf=0.3, verbose=False)[0]
            if res_p.boxes is not None:
                for b in res_p.boxes.xyxy.detach().cpu().numpy().tolist():
                    duplicate = False
                    for oid, lbl in class_of.items():
                        if lbl != "player": continue
                        prev_mask = ema_masks.get(oid)
                        if prev_mask is None: continue
                        ys, xs = np.where(prev_mask > 0.5)
                        if xs.size == 0: continue
                        prev_box = [xs.min(), ys.min(), xs.max(), ys.max()]
                        if iou(prev_box, b) > 0.3:
                            duplicate = True
                            break
                    if not duplicate:
                        new_oid = seed_object(fi, b, "player", use_box=True)
                        print(f"✅ New player added {new_oid}")

        # ---------------- Ball Correction ----------------
        if fi % reseed_interval == 0:
            res_b = ball_model(bgr, conf=0.08, iou=0.4, imgsz=IMGSZ_B,
                               agnostic_nms=True, verbose=False)[0]
            if res_b.boxes is not None and len(res_b.boxes) > 0:
                b = res_b.boxes.xyxy[int(res_b.boxes.conf.argmax())].cpu().numpy().tolist()
                ball_oid = [oid for oid,lbl in class_of.items() if lbl=="ball"]
                if ball_oid:
                    oid = ball_oid[0]
                    x1,y1,x2,y2 = b
                    box_t = torch.tensor([[x1,y1,x2,y2]], device=dev, dtype=torch.float32)
                    xc, yc = (x1+x2)/2, (y1+y2)/2
                    pts  = torch.tensor([[xc,yc]], device=dev, dtype=torch.float32)
                    labs = torch.tensor([1],       device=dev, dtype=torch.long)
                    pred.add_new_points_or_box(
                        state, fi, oid,
                        points=pts, labels=labs, box=box_t,
                        normalize_coords=True
                    )
                    ball_hist.append((fi, xc, yc))

        # ---------------- 마스크 시각화 ----------------
        for oid, m in zip(obj_ids, masks):
            oid = int(oid)
            m = m.detach().cpu().numpy().squeeze()
            if m.ndim != 2: continue
            m = (m > 0).astype(np.uint8)*255
            if class_of.get(oid) == "ball":
                m = cv2.morphologyEx(m, cv2.MORPH_OPEN, np.ones((2,2), np.uint8))
            else:
                m = cv2.morphologyEx(m, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
                m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
                m = cv2.GaussianBlur(m,(3,3),0)

            cur = (m>127).astype(np.float32)
            prev = ema_masks.get(oid)
            keep = 0.35 if class_of.get(oid)=="ball" else 0.7
            ema = cur if prev is None else keep*prev+(1-keep)*cur
            ema_masks[oid]=ema; m_bin=ema>0.5

            col = (0,0,255) if class_of.get(oid)=="ball" else color_for(oid)
            overlay[m_bin]=col
            ys,xs=np.where(m_bin)
            if xs.size:
                x1,x2,y1,y2=xs.min(),xs.max(),ys.min(),ys.max()
                cv2.rectangle(overlay,(x1,y1),(x2,y2),col,2)
                cv2.putText(overlay,f"{class_of.get(oid)} {oid}",(x1,max(0,y1-6)),
                            cv2.FONT_HERSHEY_SIMPLEX,0.6,col,2)
                if class_of.get(oid)=="ball":
                    cx,cy=(x1+x2)/2.0,(y1+y2)/2.0
                    ball_hist.append((fi,cx,cy))

        bgr=cv2.addWeighted(overlay,0.35,bgr,0.65,0)
        out.write(bgr)

cap.release(); out.release()
print("✅ saved:", OUT_PATH)

Obtaining file:///content/drive/MyDrive/sam2
